### Chapter 3　言語処理における深層学習の基礎

---
### 3.1　準備：記号の世界とベクトルの世界の橋渡し
離散的な記号である文字や単語などを連続する実数値のベクトルへ変換する方法の１つとして  
one-hotベクトルが用いられる。

語彙 $\mathcal{V}$ があり、単語数は $|\mathcal{V}|$ である。  
ここに含まれる各単語に１から $|\mathcal{V}|$ までの単語番号を割り振ったとする。  
$i$ 番目の単語の番号が $n$ のとき、この単語を表すベクトル $\mathbf{x}_{i}$ は  
$n$ 番目の要素が１で残りが０であるような one-hotベクトルで表すこととする。

単語などをニューラルネットで処理する場合は、例えば上記の考え方に基づいて  
変換行列 $E$ に one-hotベクトル $\mathbf{x}$ をかけて埋め込みベクトル $\mathbf{Ex} = \bar{\mathbf{x}}$ を取得する。  
この行列 $E$ は各列が一つの単語に対応し、これを埋め込み行列（emedding matrix）という。

---
逆にベクトルから記号へ変換する場合の典型的手法は、
語彙数 $|\mathcal{V}|$ に等しい次元数のベクトル $\mathbf{o}$ を用意し、
要素中で最大の要素番号に対応する単語番号の単語が選択されたとみなして変換することである。

スケーリング係数 $a$ を導入したソフトマックス関数

\begin{align*}
\operatorname{softmax_{a}}(\mathbf{o}) = \frac{1}{\exp (a \mathbf{o}) \cdot 1} \exp(a \mathbf{o})
\end{align*}

を考える。
式中の $\mathbf{o}$ は強化学習でいうところの逆温度パラメータである。

$\mathbf{o}$ が $|\mathcal{V}|$ 次元のベクトルであるとき、$\mathbf{x} = \operatorname{softmax_{a}}(\mathbf{o})$ の $j$ 番目の要素 $x_{j}$ は

\begin{align*}
s_{j} = \frac{\exp(ao_{j})}{\sum_{j^{\prime}=1}^{|\mathcal{V}|} \exp (ao_{j^{\prime}})} \, \forall j
\end{align*}

として計算される。  
このときソフトマックス関数のパラメータ $a$ を十分大きい値に設定した場合、  
得られるベクトル $\mathbf{x} = \operatorname{softmax_{a}}(\mathbf{o})$ は  
$\mathbf{o}$ のすべての要素のうち最大値をとる要素番号の値が１であるようなone-hotベクトルを近似する。

---
### 3.2　言語モデル
言語モデルあるいは確率的言語モデル（probabilistic language model）は  
人間が扱う自然言語で書かれた文や文書が生成される確率をモデル化したものであり、  
自然言語らしさを推定する道具となるほか、その確率分布からサンプリングすることで文や文書を生成できる。  

以降は言語モデルの基本処理単位を単語と呼称し、入出力は one-hotベクトルとする。

長さ $T$ の単語列を $\mathbf{Y}=(\mathbf{y}_{1}, \mathbf{y}_{2}, \ldots, \mathbf{y}_{T})$ とする。  
このとき単語列 $\mathbf{Y}$ を生成する確率は $P(\mathbf{Y})$である。  
ある単語列が文を構成していることを明示する仮想単語として文頭を表す BOS、文末を表す EOSを導入し、  
文を $\mathbf{Y}^{\prime} = (\mathbf{y}_{0}, \mathbf{y}_{1}, \ldots, \mathbf{y}_{T}, \mathbf{y}_{T+1})$ とする。  
このとき一般に $P(\mathbf{Y}) \neq P(\mathbf{Y}^{\prime})$ である。

言語モデルとして古典的にはNグラムモデルが最も多く使われた。  
これと区別する意味で、ニューラルネットを用いた言語モデルをニューラル言語モデルという。  

---
BOS,EOSを導入した文を $\mathbf{Y} = (\mathbf{y}_{0}, \mathbf{y}_{1}, \ldots, \mathbf{y}_{T}, \mathbf{y}_{T+1})$ と表し、文の生成確率 $P(\mathbf{Y})$ をモデル化する。  
直接モデル化は難しいため、文中の各単語の生成確率を、  
その単語の前に出現した単語(列)が与えられた条件下で予測するモデルの組み合わせとして定義されるのが一般的である。 

ここで文脈（context）を、  
　言語モデルにおいてある単語の出現確率を計算する際に用いる周囲の単語  
と定義する。

単語の位置 $t$ より前に出現した $t-a$ 単語を $\mathbf{Y}_{[a,t-1]} = (\mathbf{y}_{a}, \mathbf{y}_{a+1}, \ldots, \mathbf{y}_{t-1})$ と書き、これを文脈として扱う。  
このとき文 $\mathbf{Y}$ の出現確率 $o(\mathbf{Y})$ を各単語が生成される条件付き確率の積で次のように表す。

\begin{align*}
P(\mathbf{Y}) = P(\mathbf{y}_{0}) \prod_{t=1}^{T+1} P(\mathbf{y}_{t}|\mathbf{Y}_{[0,t-1]})
\end{align*}

例えば $\mathbf{Y}=(\mathbf{y}_{0}, \mathbf{y}_{1}, \mathbf{y}_{2})$ について、 $P(X|Y)=P(X,Y)/P(Y)$ より

\begin{align*}
P(\mathbf{Y}) &= P(\mathbf{y}_{0})P(\mathbf{y}_{1}|\mathbf{y}_{0})P(\mathbf{y}_{2}|\mathbf{y}_{1},\mathbf{y}_{0}) \\
&= P(\mathbf{y}_{0},\mathbf{y}_{1})P(\mathbf{y}_{2}|\mathbf{y}_{1},\mathbf{y}_{0}) \\
&= P (\mathbf{y}_{0},\mathbf{y}_{1},\mathbf{y}_{2})
\end{align*}

である。

条件付き確率をモデル化する。

\begin{align*}
P_{model}(\mathbf{Y}) = \prod_{t=1}^{T+1}P_{model}(\mathbf{y}_{t}|\mathbf{Y}_{[a,t-1]})
\end{align*}

このモデルにおいて、BOSを使用するなら $P(\mathbf{y}_{0})=1$ であるから $P_{model}(\mathbf{y}_{0})$ は省略する。  
また、次のように、$t-a$ 単語だけ（$\mathbf{Y}_{[a,t-1]}$）を使って条件付き確率を表現できると仮定している。

\begin{align*}
P(\mathbf{y}_{t}|\mathbf{Y}_{[0,t-1]}) \approx P_{model}(\mathbf{y}_{t}|\mathbf{Y}_{[a,t-1]})
\end{align*}

たとえば $a=t-4$ とするとき、N=5のNグラム言語モデルと同様となる。

---
#### 順伝播型ニューラル言語モデル（FFNN言語モデル）
前 $C$ 単語だけを入力とし、FFNNを用いて $t$ 番目の単語の出現確率をモデル化する。

\begin{align*}
P_{ffnnlm}(\mathbf{Y}) = \prod_{t=1}^{T+1}P(\mathbf{y}_{t}|\mathbf{Y}_{[t-C,t-1]})
\end{align*}

FFNNへの入力は前 $C$ 単語分のone-hotベクトルを連結した $\mathbf{Y}_{[t-C,t-1]}$ となる。

\begin{align*}
\mathbf{Y}_{[t-C,t-1]} &= (\mathbf{y}_{t-C},\mathbf{y}_{t-C+1},\ldots,\mathbf{y}_{t-1}) \\
&= (\mathbf{y}_{t^{\prime}})_{t^{\prime}=t-C}^{t-1}
\end{align*}

例えば、３層FFNNでの処理の流れは次の通り。

\begin{align*}
\text{(embedding vector) : } \mbox{ }& \mathbf{e}_{k}=\mathbf{Ey}_{k} \, \forall k \in \{t-C, \ldots, t-1\} \\
\text{(concatenate) : } \mbox{ }& \tilde{\mathbf{y}}_{t}=\operatorname{concat} ( \mathbf{e}_{t-C}, \ldots, \mathbf{e}_{t-1}) \\
\text{(hidden layer) : } \mbox{ }& \mathbf{h}_{t}=\operatorname{tanh}(\mathbf{W}^{(l)}\tilde{\mathbf{y}}_{t} + \mathbf{b}^{(l)}) \\
\text{(output layer) : } \mbox{ }& \mathbf{o}_{t}=\mathbf{W}^{(o)}\mathbf{h}_{t} + \mathbf{b}^{(o)} \\
\text{(probability) : } \mbox{ }& \mathbf{p}_{t} = \operatorname{softmax}(\mathbf{o}_{t}) \\
\text{(extract }P(\mathbf{y}_{t})\text{) : }\mbox{ }& P(\mathbf{y}_{t}|\mathbf{Y}_{[t-C,t-1]}) = \mathbf{p}_{t} \cdot \mathbf{y}_{t}
\end{align*}

---
#### 再帰ニューラル言語モデル（RNN言語モデル）
入力ベクトル列 $\mathbf{X}$ は  
時刻 $t$ より前の単語列を one-hotベクトル表現したベクトル列 $\mathbf{Y}_{[0,t-1]} = (\mathbf{y}_{0}, \mathbf{y}_{1}, \ldots, \mathbf{y}_{t-1})$ となる。  
これを $\mathbf{Y}_{<t} = \mathbf{Y}_{[0,t-1]}$ と略記する。

RNN言語モデルによる文 $\mathbf{Y}$ の生成確率 $P_{rnnlm}(\mathbf{Y})$ は次の通り。

\begin{align*}
P_{rnnlm}(\mathbf{Y}) = \prod_{t=1}^{T+1} P(\mathbf{y}_{t}|\mathbf{Y}_{<t})
\end{align*}

１層RNNでの処理の流れは次の通り。

\begin{align*}
\text{(embedding vector) : } \mbox{ }& \bar{\mathbf{y}}_{t}=\mathbf{Ey}_{t-1} \\
\text{(hidden layer) : } \mbox{ }& \mathbf{h}_{t}=\operatorname{tanh} \left( \mathbf{W}^{(l)} \left[ \begin{array}\bar{\mathbf{y}}_{t} \\ \mathbf{h}_{t-1} \end{array}  \right] + \mathbf{b}^{(l)} \right) \\
\text{(output layer) : } \mbox{ }& \mathbf{o}_{t}=\mathbf{W}^{(o)}\mathbf{h}_{t} + \mathbf{b}^{(o)} \\
\text{(probability) : } \mbox{ }& \mathbf{p}_{t} = \operatorname{softmax}(\mathbf{o}_{t}) \\
\text{(extract }P(\mathbf{y}_{t})\text{) : }\mbox{ }& P(\mathbf{y}_{t}|\mathbf{Y}_{<t}) = \mathbf{p}_{t} \cdot \mathbf{y}_{t}
\end{align*}

---
#### パープレキシティ（perplexity; PPL）
評価用のデータセットを $\mathcal{D} = \{ \mathbf{Y}^{(n)} \} _{n=1}^{|\mathcal{D}|}$、総単語数を $N$、  
$n$ 番目の系列の長さを $T^{(n)}$ とする。  
このときパープレキシティを次式で計算する。

\begin{align*}
&b^{z} &| \, z=-\frac{1}{N} \sum_{n=1}^{|\mathcal{D}|} \sum_{t=1}^{T^{(n)}+1} \log_{b} P_{model} ( \mathbf{y}_{t}^{(n)}, \mathbf{Y}_{[a,t-1]}^{(n)})
\end{align*}

対数の底 $b$ としては $2$ や $e$ が使われる。  
$P_{model} ( \mathbf{y}_{t}^{(n)}, \mathbf{Y}_{[a,t-1]}^{(n)})$ の値域は $[0,1]$ であるから、その対数の値域は $[-\infty, 0]$、よって $z$ は非負の値となる。

パープレキシティは次の単語を予測する確率分布のばらつきを評価しているとみなすことができる。  
（モデルに従って正解を選ぶための困難さを示す）  
よい言語モデルの確率分布はデータと一致する次の単語にだけ高い確率を与え、ばらつきの小さい分布となる。

また、パープレキシティの計算はモデル化の単位によらず計算可能であるので、  
例えば文字単位の言語モデルを単語数に対する平均パープレキシティで評価することができる。

---
#### 言語モデルからの文生成
記号 $\sim$ を確率モデルからサンプリングする処理を示すものとするとき、  
言語モデル $P_{model}(\mathbf{y}_{t}|\mathbf{Y}_{[a,t-1]})$ からサンプリングされた $\hat{\mathbf{Y}}=(\mathbf{y}_{0}=BOS, \hat{\mathbf{y}}_{1}, \ldots, \hat{\mathbf{y}}_{\hat{T}}, \hat{\mathbf{y}}_{\hat{T+1}})$ を次のように定義する。

\begin{align*}
\hat{\mathbf{y}}_{t} \sim P_{model}(\mathbf{y}_{t}, \hat{\mathbf{Y}}_{[a,t-1]})
\end{align*}

これは文頭から順に次の単語を予測する処理ととらえることができ、  
文の終わりを示すEOSが予測されたところでサンプリングを終了する。

---
#### 言語モデルの単位
単位としては単語のほか、文字、バイト対符号化なども用いられる。

---
### 3.3　分散表現 (distributed representation)
単語など自然言語処理で扱う記号は人間が恣意的に定義したものであり、  
記号そのものの情報から記号間の類似度や関連性を直接計算することが難しい。

**離散オブジェクト**を次のように定義する。  
　人や物の名前、概念のように、物理的に計測できる量を伴わず、  
　通常、記号を用いて離散的に表現するもの。

**分散表現**はNLPの文脈では、一般的に次の意味で用いられる。  
　任意の離散オブジェクトの集合 $\mathcal{V}$ に対して、各離散オブジェクト $v \in \mathcal{V}$ に  
　それぞれ $D$ 次元ベクトルを割り当て、離散オブジェクトを $D$ 次元ベクトルで表現したもの。
 
分散表現への変換処理は、離散オブジェクトをベクトル空間内に埋め込む操作と捉えることもできる。   
ベクトル空間では距離や演算が定義できることから、    
これを離散オブジェクト間の類似度や関連性を計算する道具として用いることができる。

単語に着目した分散表現を単語分散表現、あるいは単語埋め込みと呼ぶこともある。

---
#### 歴史的背景
・認知心理学や神経科学の分野では、脳のモデル化の方法論として分散表現が考えられた。  
　分散表現の対義語として局所表現（one-hotベクトルなどが該当）が用いられる。
 
・自然言語処理研究では、単語の持つ意味（semantics）を扱う方法の１つとして分布仮説が提案された。  
　この仮説は、単語の意味は出現した際の周囲の単語によって決まるという考え方である。  
　その手法としては主成分分析や潜在ディレクレ配分などがあり、  
　分布仮説に基づいた方法論で得られた単語のベクトル表現を分散表現（distributional representation）と呼ぶ。
 
・ニューラル言語モデルでは両分野の概念が同時に利用されている。

---
#### 分散表現の獲得方法
１．ニューラル言語モデルを学習することで、変換行列が分散表現に相当する値へ更新される

２．対数線形性モデル（log-bilinear model; LBL model）を用いる  
　word2vecおよび対数線形性モデルと呼ばれるほぼ同じモデルが同時期に提案された。  
　（word2vecというツールにはskip-gramモデルおよびCBoWモデルが実装されている）
 
 

---
#### skip-gram および CBoWについて
語彙を $\mathcal{V}$、$i$ 番目の単語番号の入力単語 $\mathbf{x}_{i}$ に割り当てる単語埋め込みベクトルを $\mathbf{e}_{i}$、  
$j$ 番目の単語番号の出力単語 $\mathbf{y}_{j}$ に割り当てる単語埋め込みベクトルを $\mathbf{o}_{j}$ で表す。  
ここで $1 \leq i \leq |\mathcal{V}|$、$1 \leq j \leq |\mathcal{V}|$ である。

$\mathbf{e}_{i}$ と $\mathbf{o}_{j}$ を $1$ から $|\mathcal{V}|$ まで並べて行列表示とみなすと、 $\mathbf{E} = (\mathbf{e}_{i})_{i=1}^{|\mathcal{V}|}$、$\mathbf{O} = (\mathbf{o}_{j})_{j=1}^{|\mathcal{V}|}$ と書ける。  
ここで $\mathbf{e}_{i}$ と $\mathbf{o}_{j}$ は $D$ 次元の列ベクトルとする。

**< CBoW >**  
入力単語のリストを $\mathcal{H}$ とする。   
$\mathcal{H}$ が与えられた時に $j$ 番目の出力単語 $\mathbf{y}_{j}$ が出力される確率を以下の式で定義する。

\begin{align*}
P(\mathbf{y}_{i}|\mathcal{H}) &= \frac{\exp(\phi (\mathcal{H},\mathbf{y}_{j}))}{\sum_{\mathbf{y}_{j^{\prime}} \in \mathcal{V}} \exp (\phi (\mathcal{H}, \mathbf{y}_{j^{\prime}}))} \\
\phi(\mathcal{H},\mathbf{y}_{j}) &= \sum_{i:\mathbf{x}_{i}\in\mathcal{H}}(\mathbf{Ex}_{i}) \cdot ( \mathbf{Oy}_{j}) \\
&= \sum_{i:\mathbf{x}_{i}\in\mathcal{H}} \mathbf{e}_{i} \cdot \mathbf{o}_{j}
\end{align*}

**< skip-gram >**  
入力語彙 $\mathcal{V}$ 中の $i$ 番目の単語 $\mathbf{x}_{i}$ が与えられた時に  
出力語彙 $\mathcal{V}$ 中の $j$ 番目の単語 $\mathbf{y}_{j}$ が出力される確率を以下の式で定義する。

\begin{align*}
P(\mathbf{y}_{i}|\mathcal{H}) &= \frac{\exp(\phi (\mathcal{x}_{i},\mathbf{y}_{j}))}{\sum_{\mathbf{y}_{j^{\prime}} \in \mathcal{V}} \exp (\phi (\mathcal{x}_{i}, \mathbf{y}_{j^{\prime}}))} \\
\phi(\mathcal{x}_{i},\mathbf{y}_{j}) &= (\mathbf{Ex}_{i}) \cdot ( \mathbf{Oy}_{j}) \\
&= \mathbf{e}_{i} \cdot \mathbf{o}_{j}
\end{align*}


上の式から、skip-gramは $\mathcal{H} = \{ \mathbf{x}_{i} \}$ であるとき（入力単語リスト長が常に１）のCBoWと見なせることが分かる。  
また、対数双線形モデルにてCBoWやskip-gramを統一的に記述できることが分かっている。  

対数双線形モデルの式はFFNNの出力層と同じ形のため、  
これを１層NNによるニューラル言語モデルとみなして実装することができる。

訓練データを $\mathcal{D} = \{ (\mathcal{H}^{(n)}, \mathbf{y}_{j}^{(n)})\}_{n=1}^{N}$ とするとき、学習時の目的関数は次の通り。

\begin{align*}
L(\mathbf{E}, \mathbf{O}|\mathcal{D}) &= - \sum_{(\mathcal{H},\mathbf{y}_{j})\in\mathcal{D}} \log (P(\mathbf{y}_{j}|\mathcal{H})) \\
&= - \sum_{(\mathcal{H},\mathbf{y}_{j})\in\mathcal{D}} \phi (\mathcal{H},\mathbf{y}_{j}) + \sum_{(\mathcal{H},\mathbf{y}_{j})\in\mathcal{D}} \log \, \left( \sum_{\mathbf{y}_{j^{\prime}}\in\mathcal{V}} \exp(\phi (\mathcal{H},\mathbf{y}_{j^{\prime}})) \right)
\end{align*}

この数式は語彙 $\mathcal{V}$ 全てに基づいて計算されることから負荷が高く、  
階層的ソフトマックス等を用いた対処が行われる。

---
#### 負例サンプリングによる獲得方法
負例サンプリングによる学習は、与えられた入出力の組が  
実データの確率分布から取得されたデータか、別の確率分布から生成されたデータかの  
２クラスを判別する識別モデルの学習に相当する。

$P((\mathcal{H},\mathbf{y}_{j}) \sim P^{\mathcal{D}})$ を、学習用の１つのデータ $(\mathcal{H},\mathbf{y}_{j})$ が  
実際の訓練データ $\mathcal{D}$ を生成する確率分布 $P^{\mathcal{D}}$ から生成された場合を表す確率モデルとし、  
以下のようにモデル化する。

\begin{align*}
P((\mathcal{H},\mathbf{y}_{j}) \sim P^{\mathcal{D}}) = \frac{1}{1+\exp(-\phi (\mathcal{H},\mathbf{y}_{j}))}
\end{align*}

同様に、 $P((\mathcal{H},\mathbf{y}_{j}) \sim P^{\mathcal{D}^{\prime}})$ を $(\mathcal{H},\mathbf{y}_{j})$ が  
ノイズデータ $\mathcal{D}^{\prime}$ を生成する確率分布 $P^{\mathcal{D}^{\prime}}$ から生成された場合を表す確率モデルとする。

このとき、実データとノイズの識別問題の対数尤度関数は次の通り。

\begin{align*}
L(\mathbf{E}, \mathbf{O}|\mathcal{D}) &= - \sum_{(\mathcal{H},\mathbf{y}_{j})\in\mathcal{D}} \log (P((\mathcal{H},\mathbf{y}_{j}) \sim P^{\mathcal{D}})
+ \sum_{(\mathcal{H},\mathbf{y}_{j})\in\mathcal{D}^{\prime}} \log (P((\mathcal{H},\mathbf{y}_{j}) \sim P^{\mathcal{D}^{\prime}})
\end{align*}

これら２式より

\begin{align*}
L(\mathbf{E}, \mathbf{O}|\mathcal{D}) &= \sum_{(\mathcal{H},\mathbf{y}_{j})\in\mathcal{D}} \log (1+\exp(-\phi (\mathcal{H},\mathbf{y}_{j})))
- \sum_{(\mathcal{H},\mathbf{y}_{j})\in\mathcal{D}^{\prime}} \log (1+\exp(-\phi (\mathcal{H},\mathbf{y}_{j})))
\end{align*}

となる。

---
### 3.4　系列変換モデル